O objetivo é Extrair os tickers de TODOS os Recibos de negociação que estão atrelados as ações negociadas fora do brasil, ou seja, BDRs usando web scraping e lista-los. Os dados seram extraidos do site invest news.

# 1. Importação das Bibliotecas

O projeto começa com a importação das bibliotecas necessárias para a realização do web scraping e análise de dados:

In [5]:
import pandas as pd
import yfinance as yf
from urllib import request
from urllib.request import Request, urlopen

# 2. Web Scraping com urllib

Para extrair os dados do site Invest News, utilizamos a biblioteca urllib. A URL a ser utilizada para capturar os dados é:

In [8]:
url = 'https://investnews.com.br/financas/veja-a-lista-completa-dos-bdrs-disponiveis-para-pessoas-fisicas-na-b3/?gad_source=1&gclid=Cj0KCQjwrKu2BhDkARIsAD7GBosgjpgpE9fxZjpvY4x0hMq_T3rGc9tXXGaY5vcdd8sQA9aMGyVFoSwaAr_IEALw_wcB'

Para evitar o erro 403 (que ocorre quando o servidor bloqueia o acesso por identificar um possível comportamento malicioso), o projeto se identifica como um navegador real:

In [9]:
request_site = Request(url, headers = {"User-Agent":"Mozila/5.0"})
webpage = urlopen(request_site).read()

Os dados são extraídos em formato tabular utilizando pandas:

In [10]:
df = pd.read_html(webpage)[0]
df.columns = df.iloc[0]
# eliminando a linha após utilizada para nomear os titulos do DataFrame.
df.drop([0],axis=0, inplace =True)

In [11]:
df

,EMPRESA,CÓDIGO,SETOR,PAÍS DE ORIGEM
1,3M,MMMC34,Indústria diversificada,EUA
2,AB INBEV,ABUD34,Bebidas,Bélgica
3,ABB LTD,A1BB34,Industrial,Switzerland
4,ABBOTT,ABTT34,Farmacêuticos,EUA
5,ABBVIE,ABBV34,Medicamentos e Outros Produtos,EUA
...,...,...,...,...
704,RAYTHEON TECH,RYTT34,Industrial,EUA
705,SCHLUMBERGER,SLBG34,Equipamentos e Serviços de Combustível,EUA
706,TIFFANY,TIFF34,Joalheria,EUA
707,US BANCORP,USBC34,Instituição Financeira,EUA


In [12]:
df.shape

(708, 4)

# 3. Extração dos dados de negociação pela yfinance utilizando os codigos da lista de BDRs

Com a lista de BDRs extraída, utilizamos a biblioteca yfinance para baixar os preços de fechamento de cada BDR a partir de 1º de janeiro de 2023:

In [13]:
bdrs = list(df['CÓDIGO'])
bdr_tickers = [i + '.SA' for i in bdrs]
df_bdr = yf.download(bdr_tickers, start='2023-01-01')['Close']

[*********************100%***********************]  708 of 708 completed

68 Failed downloads:
['C1SU34.SA', 'RDSA35.SA', 'Z2EN34.SA', 'E1NI34.SA', 'B1BL34.SA', 'TWTR34.SA', 'C1OU34.SA', 'L1BT35.SA', 'CRHP34.SA', 'C1HL34.SA', 'RDSA34.SA', 'W1RK34.SA', 'D1RE34.SA', 'ATVI34.SA', 'FLTC34.SA', 'F1RC34.SA', 'PTCH34.SA', 'P1IO34.SA', 'E1FX34.SA', 'X1LN34.SA', 'M1XI34.SA', 'S1GE34.SA', 'A1UA34.SA', 'F1RI34.SA', 'LBRN34.SA', 'D1IS34.SA', 'SBNY34.SA', 'A1MX34.SA', 'A2MC34.SA', 'C1XO34.SA', 'V1AR34.SA', 'N1LS34.SA', 'LSXM34.SA', 'TIFF34.SA', 'A1BM34.SA', 'C1HI34.SA', 'A2VL34.SA', 'C1ER34.SA', 'C1TX34.SA', 'E2PD34.SA', 'INGG3.SA', 'W1BK34.SA', 'NUBR33.SA', 'L1FC34.SA', 'U1NL34.SA', 'A1NT34.SA', 'CAJI34.SA', 'A1MB34.SA', 'SLBC34.SA', 'A1LX34.SA', 'BHPG34.SA', 'H1ZN34.SA', 'FBOK34.SA', 'A1BB34.SA', 'P1KI34.SA', 'C1EO34.SA', 'P1BC34.SA', 'L1BT34.SA', 'DCVY35.SA', 'LSXM35.SA', 'A1YX34.SA', 'K1CS34.SA', 'S2TO34.SA', 'L1IN34.SA', 'E1XC34MONDE.SA', 'I1NF34.SA', 'ISBC34.SA', 'S1PL34.SA']: YFTzMissingErro

# 4. Tratamento de Dados

Algumas das extrações podem falhar devido à descontinuação ou outros problemas com certos BDRs. Para lidar com isso, os valores ausentes são preenchidos com o método forward fill:

In [14]:
df_bdr = df_bdr.ffill(axis=1)

In [51]:
df_bdr

Ticker,A1AP34.SA,A1BB34.SA,A1BM34.SA,A1CR34.SA,A1DI34.SA,A1DM34.SA,A1EE34.SA,A1EG34.SA,A1EN34.SA,A1EP34.SA,...,XRXB34.SA,YUMR34.SA,Z1BH34.SA,Z1BR34.SA,Z1IO34.SA,Z1OM34.SA,Z1TO34.SA,Z1TS34.SA,Z2EN34.SA,Z2LL34.SA
Date,,,,,,,,,,,,,,,,,,,,,
2023-01-02 00:00:00+00:00,45.070000,45.070000,45.070000,62.889999,459.179993,502.160004,232.000000,27.000000,289.500000,250.750000,...,76.750000,317.269989,138.869995,45.150002,293.50,14.67,34.980000,48.549999,48.549999,13.98
2023-01-03 00:00:00+00:00,50.310001,50.310001,50.310001,64.080002,459.179993,492.429993,232.000000,27.410000,289.500000,250.750000,...,76.750000,317.269989,138.869995,46.980000,293.50,14.61,34.980000,50.139999,50.139999,14.89
2023-01-04 00:00:00+00:00,52.160000,52.160000,52.160000,65.349998,451.320007,471.190002,232.000000,27.910000,289.500000,250.750000,...,76.750000,317.269989,138.869995,48.099998,293.50,14.74,34.980000,50.340000,50.340000,14.89
2023-01-05 00:00:00+00:00,51.230000,51.230000,51.230000,64.260002,428.000000,460.130005,232.000000,26.780001,292.019989,251.899994,...,76.750000,317.269989,138.869995,45.950001,293.50,14.10,34.980000,48.799999,48.799999,14.89
2023-01-06 00:00:00+00:00,50.770000,50.770000,50.770000,63.720001,435.160004,444.250000,232.000000,27.200001,292.019989,251.899994,...,76.750000,317.269989,138.869995,45.950001,293.50,14.58,34.980000,48.500000,48.500000,14.89
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-19 00:00:00+00:00,20.540001,20.540001,20.540001,57.119999,607.109985,320.000000,222.419998,34.919998,256.450012,265.459991,...,55.919998,376.200012,155.850006,58.779999,251.75,12.92,27.030001,62.040001,62.040001,24.16
2024-08-20 00:00:00+00:00,20.879999,20.879999,20.879999,58.259998,607.109985,320.000000,222.419998,35.430000,256.450012,265.459991,...,55.919998,376.200012,155.850006,58.779999,251.75,12.92,27.030001,62.880001,62.880001,24.16
2024-08-21 00:00:00+00:00,21.180000,21.180000,21.180000,58.259998,607.109985,324.160004,222.419998,35.520000,256.450012,265.459991,...,55.919998,376.200012,155.850006,58.779999,251.75,13.10,27.030001,62.820000,62.820000,24.16


# Resultado

Ao final, o projeto gera um DataFrame contendo os preços históricos de fechamento de todos os BDRs disponíveis, que pode ser utilizado para análises mais aprofundadas ou para a construção de estratégias de investimento.